Nesta fase, irei iniciar a etapa de experimentação, onde irei testar uma série 
de modelos de classificação para encontrar o que melhor se adequa ao problema.
Vale ressaltar que apenas algumas colunas serão utilizadas para a criação dos
modelos, pois algumas só são obtidas ao fim do processo do SISU, como é o caso
da coluna "NOTA_CORTE" e "CLASSIFICACAO". Outras colunas como códigos das IES
e cursos também não serão utilizadas, pois não são relevantes para o problema.
Ao final, além do modelo, um conjunto de dados no formato .db será gerado para
ser consumido pelo app final. Segue abaixo as colunas que serão utilizadas:

- Modelo: IES, UF_CAMPUS, MUNICIPIO_CAMPUS, NOME_CURSO, GRAU, TURNO, 
TIPO_MOD_CONCORRENCIA, QT_VAGAS_CONCORRENCIA, PERCENTUAL_BONUS, PESO_L, PESO_CH,
PESO_CN, PESO_M, PESO_R, NOTA_MINIMA_L, NOTA_MINIMA_CH, NOTA_MINIMA_CN, 
NOTA_MINIMA_M, NOTA_MINIMA_R, MEDIA_MINIMA, OPCAO, NOTA_L, NOTA_CH, NOTA_CN, 
NOTA_M, NOTA_R, NOTA_L_COM_PESO, NOTA_CH_COM_PESO, NOTA_CN_COM_PESO, 
NOTA_M_COM_PESO, NOTA_R_COM_PESO, NOTA_CANDIDATO e APROVADO.

Vale ressaltar que parte das informações que serão utilizadas no Web App serão 
buscadas nos dados do SISU, como é o caso da QT_VAGAS_CONCORRENCIA, que é um 
valor que a universidade define para cada curso e não o usuário. Outras serão 
calculadas manualmente, como no caso das notas com peso.


In [1]:
import mlflow
import joblib
import optuna
import pandas as pd
import category_encoders as ce

# Preprocessing & Models
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# Métricas
from sklearn.metrics import log_loss

In [2]:
# Lendo os dados
dados_sisu_full = pd.read_parquet('../data/processed/dados_transformados.parquet/')

In [3]:
# Definindo as colunas que serão utilizadas para treinar o modelo
colunas_para_buscar = ['IES', 'UF_CAMPUS', 'MUNICIPIO_CAMPUS', 'NOME_CURSO', 
                       'GRAU', 'TURNO', 'TIPO_MOD_CONCORRENCIA', 
                       'QT_VAGAS_CONCORRENCIA', 'PERCENTUAL_BONUS', 'PESO_L', 
                       'PESO_CH', 'PESO_CN', 'PESO_M', 'PESO_R', 
                       'NOTA_MINIMA_L', 'NOTA_MINIMA_CH', 'NOTA_MINIMA_CN', 
                       'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA', 
                       'OPCAO', 'NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 
                       'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO', 
                       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
                       'NOTA_CANDIDATO', 'APROVADO']

In [4]:
# Filtrando as colunas
dados_sisu = dados_sisu_full[colunas_para_buscar]

In [5]:
# Verificando o balanceamento da variável alvo
dados_sisu['APROVADO'].value_counts(normalize=True)

N    0.898245
S    0.101755
Name: APROVADO, dtype: float64

In [7]:
# Define o local para salvar os experimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

<Experiment: artifact_location='/home/daniel/Documents/sisu_analysis/notebooks/../mlruns/980913714926035010', creation_time=1702469991297, experiment_id='980913714926035010', last_update_time=1702469991297, lifecycle_stage='active', name='Comparando modelos', tags={}>

In [6]:
# Dividindo os dados em variaveis explicativas e variavel alvo
x = dados_sisu.drop(columns=['APROVADO'])
y = dados_sisu['APROVADO'].map({'S': 1, 'N': 0})

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.45, random_state=42, stratify=y)

# Dividindo os dados em teste e dev
x_teste, x_dev, y_teste, y_dev = train_test_split(x_teste, y_teste, test_size=0.5, random_state=42, stratify=y_teste)

# Dividindo os dados em dev e calibração
x_dev, x_calib, y_dev, y_calib = train_test_split(x_dev, y_dev, test_size=0.5, random_state=42, stratify=y_dev)

In [ ]:
# Criando um scaler padrão
scale = y_treino.value_counts()[0] / y_treino.value_counts()[1] 

# Criando dicionário com os modelos
dict_models_scale_sensitive_cw = {"LR": LogisticRegression(random_state=200, 
                                                           class_weight='balanced')}

dict_models_scale_sensitive_no_cw = {"LR": LogisticRegression(random_state=200)}

dict_models_tree_based_cw = {"LGBM": LGBMClassifier(is_unbalance=True,
                                                 random_state=200),
                          "XGB": XGBClassifier(scale_pos_weight=scale,
                                               random_state=200),
                          "CTBC": CatBoostClassifier(auto_class_weights='Balanced',
                                                     random_state=200)}

dict_models_tree_based_no_cw = {"LGBM": LGBMClassifier(random_state=200),
                          "XGB": XGBClassifier(random_state=200),
                          "CTBC": CatBoostClassifier(random_state=200)}

# Criando dicionário com os encoders
dict_encoders = {"OHE": OneHotEncoder(drop='first'),
                 "TE": ce.TargetEncoder(),
                 "BE": ce.BinaryEncoder(),
                 "ME": ce.MEstimateEncoder(),
                 "WOE": ce.WOEEncoder(),
                 "CE": ce.CatBoostEncoder(),
                 "GE":ce.GrayEncoder()}

dict_scalers = {"SS": StandardScaler()}

# Criando dicionário com os transformers
dict_transformers = {"PT": PowerTransformer()}

In [7]:
# Definindo as folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=200)

# Retornando os nomes das colunas com mais de 25 valores únicos
cat_cols = x_treino.select_dtypes(include='object').columns
high_dim_cols = cat_cols[x_treino[cat_cols].nunique() > 25]

# Retornando os nomes das colunas com menos de 25 valores únicos
cat_cols = [col for col in cat_cols if col not in high_dim_cols]

# Buscando as colunas numéricas
num_cols = x_treino.select_dtypes(include=['int', 'float']).columns

In [ ]:
## Iniciando os experimentos sem transformers e com class_weight
#for tag, model in dict_models_scale_sensitive_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_CW_{tag_encoder}_{tag_scaler}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 pipe_num = Pipeline([('scaler', scaler)])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('num', pipe_num, num_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()           
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)

In [ ]:
## Iniciando os experimentos sem transformers e sem class_weight
#for tag, model in dict_models_scale_sensitive_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_NO_CW_{tag_encoder}_{tag_scaler}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 pipe_num = Pipeline([('scaler', scaler)])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('num', pipe_num, num_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()           
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)

In [ ]:
## Iniciando os experimentos com transformers e sem class_weight
#for tag, model in dict_models_scale_sensitive_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            for tag_transformer, transformer in dict_transformers.items():
#            
#                # Gerando a tag de identificação do modelo
#                nome_modelo = f'{tag}_NO_CW_{tag_encoder}_{tag_scaler}_{tag_transformer}'
#
#                with mlflow.start_run(run_name=nome_modelo):
#
#                     # Criando os pipeline com os transformers
#                     pipe_cat = Pipeline([('encoder', encoder)])
#                     pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                     pipe_num = Pipeline([('scaler', scaler),
#                                          ('transformer', transformer)])
#
#                     # Criando o transformador
#                     transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                     ('num', pipe_num, num_cols),
#                                                     ('high_dim', pipe_high_dim, high_dim_cols)])
#
#                     # Criando o pipeline final
#                     pipe = Pipeline([('transformer', transformer),
#                                     ('model', model)])
#
#                     # Executando o cross validation
#                     cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#
#                     # Calculando a média das métricas
#                     mean_score = cross_val_scores.mean()         
#
#                     # Salvando a métrica da folder 1
#                     mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#
#                     # Salvando a métrica da folder 2
#                     mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#
#                     # Salvando a métrica da folder 3
#                     mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#
#                     # Salvando a métrica da folder 4
#                     mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#
#                     # Salvando a métrica da folder 5
#                     mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#
#                     # Salvando as métricas
#                     mlflow.log_metric('log_loss_mean', mean_score)

In [ ]:
## Iniciando os experimentos com transformers e com class_weight
#for tag, model in dict_models_scale_sensitive_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#        for tag_scaler, scaler in dict_scalers.items():
#            for tag_transformer, transformer in dict_transformers.items():
#            
#                # Gerando a tag de identificação do modelo
#                nome_modelo = f'{tag}_CW_{tag_encoder}_{tag_scaler}_{tag_transformer}'
#
#                with mlflow.start_run(run_name=nome_modelo):
#
#                     # Criando os pipeline com os transformers
#                     pipe_cat = Pipeline([('encoder', encoder)])
#                     pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                     pipe_num = Pipeline([('scaler', scaler),
#                                          ('transformer', transformer)])
#
#                     # Criando o transformador
#                     transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                     ('num', pipe_num, num_cols),
#                                                     ('high_dim', pipe_high_dim, high_dim_cols)])
#
#                     # Criando o pipeline final
#                     pipe = Pipeline([('transformer', transformer),
#                                     ('model', model)])
#
#                     # Executando o cross validation
#                     cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#
#                     # Calculando a média das métricas
#                     mean_score = cross_val_scores.mean()         
#
#                     # Salvando a métrica da folder 1
#                     mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#
#                     # Salvando a métrica da folder 2
#                     mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#
#                     # Salvando a métrica da folder 3
#                     mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#
#                     # Salvando a métrica da folder 4
#                     mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#
#                     # Salvando a métrica da folder 5
#                     mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#
#                     # Salvando as métricas
#                     mlflow.log_metric('log_loss_mean', mean_score)

In [ ]:
## Iniciando os experimentos sem transformers e com class_weight
#for tag, model in dict_models_tree_based_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_CW_{tag_encoder}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()         
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)
#

In [ ]:
## Iniciando os experimentos sem transformers e sem class_weight
#for tag, model in dict_models_tree_based_no_cw.items():
#    for tag_encoder, encoder in dict_encoders.items():
#            
#            # Gerando a tag de identificação do modelo
#            nome_modelo = f'{tag}_NO_CW_{tag_encoder}'
#            
#            with mlflow.start_run(run_name=nome_modelo):
#                 
#                 # Criando os pipeline com os transformers
#                 pipe_cat = Pipeline([('encoder', encoder)])
#                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#                 
#                 # Criando o transformador
#                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
#                 
#                 # Criando o pipeline final
#                 pipe = Pipeline([('transformer', transformer),
#                                 ('model', model)])
#                 
#                 # Executando o cross validation
#                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='neg_log_loss')
#                 
#                 # Calculando a média das métricas
#                 mean_score = cross_val_scores.mean()         
#                 
#                 # Salvando a métrica da folder 1
#                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
#                 
#                 # Salvando a métrica da folder 2
#                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
#                
#                 # Salvando a métrica da folder 3
#                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
#                
#                 # Salvando a métrica da folder 4
#                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
#                
#                 # Salvando a métrica da folder 5
#                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
#                 
#                 # Salvando as métricas
#                 mlflow.log_metric('log_loss_mean', mean_score)
#

In [ ]:
# Especificando as colunas para retornar
colunas = ['tags.mlflow.runName', 'metrics.log_loss_mean']

# Buscando os melhores modelos
mlflow.search_runs(order_by=['metrics.log_loss_mean DESC'], max_results=15)[colunas]

Os modelos sem Class_Weight foram os melhores classificadores. Dentre os 
melhores, é perceptível a baixa diferença entre os modelos. Dado a isso, iremos
selecionar os dois melhores modelos para a tunagem de hiperparâmetros, que são 
a Regressão Logística e o CatBoost. Os dois serão testados em um ensemble.

Como a natureza da solução exige respostas rápidas, a Regressão Logística terá 
preferência sobre o CatBoost e o Ensemble de ambos.

### Regressão Logística

In [ ]:
## Criando função para tunar o modelo
#def objective(trial):
#
#    params = {
#        'C': trial.suggest_float('C', 1e-4, 1e+4, log=True),
#        'penalty': trial.suggest_categorical('penalty', [None, 'l2']),
#        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga', 'newton-cholesky']),
#        'max_iter': trial.suggest_int('max_iter', 50, 1000),
#        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
#        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
#        'random_state': 200
#    }
#    
#    # Criando os pipeline com os transformers
#    pipe_cat = Pipeline([('encoder', ce.BinaryEncoder())])
#    pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#    pipe_num = Pipeline([('scaler', StandardScaler())])
#    
#    # Criando o transformador
#    transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                    ('num', pipe_num, num_cols),
#                                    ('high_dim', pipe_high_dim, high_dim_cols)])
#    
#    # Criando o pipeline final
#    pipe = Pipeline([('transformer', transformer),
#                    ('model', LogisticRegression(**params))])
#
#    # Treinando o modelo com os dados de treino
#    pipe.fit(x_treino, y_treino)
#   
#    logloss = log_loss(y_dev, pipe.predict_proba(x_dev))
#    
#    return logloss
#
## Criando o estudo de otimização
#study = optuna.create_study(direction = 'minimize')
#study.optimize(objective, n_trials = 15)

In [ ]:
# O resultado do hiperparâmetro otimizado pode ser diferente a cada execução.
# Resultado atingido: 0.23

# Checando os melhores parâmetros
#study.best_params

In [11]:
# Criando os pipeline com os transformers
pipe_cat = Pipeline([('encoder', ce.BinaryEncoder())])
pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
pipe_num = Pipeline([('scaler', StandardScaler())])

# Capturando os indices das colunas
cat_cols_index = [x_treino.columns.get_loc(col) for col in cat_cols]
num_cols_index = [x_treino.columns.get_loc(col) for col in num_cols]
high_dim_cols_index = [x_treino.columns.get_loc(col) for col in high_dim_cols]

# Criando o transformador
transformer = ColumnTransformer([('cat', pipe_cat, cat_cols_index),
                                ('num', pipe_num, num_cols_index),
                                ('high_dim', pipe_high_dim, high_dim_cols_index)])

# Criando o pipeline final
best_lr = Pipeline([('transformer', transformer),
                ('model', LogisticRegression(C=0.08250109742237544,
                                             penalty='l2',
                                             solver='newton-cholesky',
                                             max_iter=244,
                                             fit_intercept=False,
                                             class_weight=None,
                                             random_state=200))])

best_lr.fit(x_treino.values, y_treino.values)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   BinaryEncoder())]),
                                                  [4, 5, 6, 9, 10, 11, 12, 13,
                                                   20]),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [7, 8, 14, 15, 16, 17, 18, 19,
                                                   21, 22, 23, 24, 25, 26, 27,
                                                   28, 29, 30, 31]),
                                                 ('high_dim',
                                                  Pipeline(steps=[('encoder',
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  [0, 1, 2, 3])])),
                ('model',
                 LogisticRegression(C=0.08250109742237544, fit_intercept=False,
                                    max_iter=244, random_state=200,
                                    solver='newton-cholesky'))])

### CatBoost

In [ ]:
## Criando função para tunar o modelo
#def objective(trial):
#
#    params = {
#        'objective': 'Logloss',
#        'eval_metric': 'Logloss',
#        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
#        'depth': trial.suggest_int('depth', 3, 10),
#        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
#        'subsample': trial.suggest_float('subsample', 0.5, 1),
#        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1),
#        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10),
#        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#        'random_state': 200
#    }
#    
#    pipe_cat = Pipeline([('encoder', ce.GrayEncoder())])
#    pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
#
#    # Criando o transformador
#    transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
#                                    ('high_dim', pipe_high_dim, high_dim_cols)])
#
#    # Criando o pipeline final
#    pipe = Pipeline([('transformer', transformer),
#                    ('rf', CatBoostClassifier(**params))])
#
#    # Treinando o modelo com os dados de treino
#    pipe.fit(x_treino, y_treino)
#   
#    logloss = log_loss(y_dev, pipe.predict_proba(x_dev))
#    
#    return logloss
#
## Criando o estudo de otimização
#study = optuna.create_study(direction = 'minimize')
#study.optimize(objective, n_trials = 15)

Não foi registrado um aumento significativa com a tunagem, por isso, o modelo
será o padrão.

# Testando e retreianando o modelo com todos os dados

Como o Catboost não obteve um aumento significativo com a tunagem, iremos
seguir apenas com a Regressão Logística. Além disso, por não necessitar de
calibração de probabilidade, essa parte será pulada. Entretando, como os dados 
para calibração já foram divididos, iremos utiliza-los para avaliação extra.

In [12]:
# Avaliando a melhor Regressão Logística nos dados de teste
log_loss(y_teste.values, best_lr.predict_proba(x_teste.values))

0.2351030861363363

In [13]:
# Avaliando a melhor Regressão Logística nos dados de calibração
log_loss(y_calib.values, best_lr.predict_proba(x_calib.values))

0.23674439775667952

In [14]:
# Retreinando o modelo com todos os dados
best_lr.fit(x.values, y.values)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   BinaryEncoder())]),
                                                  [4, 5, 6, 9, 10, 11, 12, 13,
                                                   20]),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [7, 8, 14, 15, 16, 17, 18, 19,
                                                   21, 22, 23, 24, 25, 26, 27,
                                                   28, 29, 30, 31]),
                                                 ('high_dim',
                                                  Pipeline(steps=[('encoder',
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  [0, 1, 2, 3])])),
                ('model',
                 LogisticRegression(C=0.08250109742237544, fit_intercept=False,
                                    max_iter=244, random_state=200,
                                    solver='newton-cholesky'))])

In [20]:
# Salvando o modelo
joblib.dump(best_lr, '../models/best_lr.pkl')

['../models/best_lr.pkl']

# Salvando parte dos dados com informações relevantes para o Web App
Por fim, a parte dos dados que contém informações relevantes para o Web App
será salva em um arquivo .db para ser consumido pelo app. Colunas referentes as
notas dos candidatos serão removidas, pois serão fornecidas pelo usuário.

In [ ]:
# Adicionando uma coluna extra na lista
colunas_para_buscar.append('MOD_CONCORRENCIA')

# Selecionando as colunas
dados_to_save = dados_sisu_full[colunas_para_buscar]
dados_to_save = dados_to_save.drop(['NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 'NOTA_R', 'NOTA_CANDIDATO'], axis = 1)

In [ ]:
# Salvando os dados no formato .db
dados_to_save.to_sql('dados_to_web', 'sqlite:///../data/processed/dados_to_web.db', index=False, if_exists='replace')